In [1]:
import pandas as pd
import json 
import pandas as pd 
import numpy as np
import collections
from collections import defaultdict
import re
import random


In [2]:
class Domain_question_generation:
    
    def __init__(self, data):
        with open(data, 'r') as f:
            self.data = f.readlines()
        self.data = [json.loads(x) for x in self.data]
        optimal_sequence = []
        for timestep in self.data:
            for action,value in timestep.items():
                if value['part_of_plan?'] == True:
                    optimal_sequence.append(action)
        self.optimal_sequence = optimal_sequence
        self.executable_actions = self.extract_executable_actions()
        self.inexecutable_actions = self.extract_inexecutable_actions()
        self.fluents_from_executable_actions = self.extract_fluents_from_executable_actions()
        self.fluents_from_optimal_sequence = self.extract_fluents_from_optimal_sequence()
                    
    def extract_executable_actions(self):
        """This function extracts the executable actions
            from the entire plan which are not included
            in the optimal sequence"""
            
        exeutable_actions = []
        for timestep in self.data: #timestep is a dictionary with action as key and value as another dictionary
            timestep_executable_actions = [] 
            for action, value in timestep.items(): 
                if value['part_of_plan?'] == False and value['feasible?']==True and len(value['fluents'])>0:
                    timestep_executable_actions.append(action)  
            exeutable_actions.append(timestep_executable_actions)
        return exeutable_actions

    def extract_inexecutable_actions(self):
        """This function extracts the inexecutable actions from the entire plan"""
        
        inexecutable_actions = []
        for timestep in self.data:
            timestep_inexecutable_actions = []        
            for action, value in timestep.items():
                if value['part_of_plan?'] == False and value['feasible?']==False and len(value['fluents'])==0:
                    timestep_inexecutable_actions.append(action)   
            inexecutable_actions.append(timestep_inexecutable_actions)
        return inexecutable_actions    

    def extract_fluents_from_executable_actions(self):
        """This function extracts the fluents 
            from the executable actions which
            are not in the optimal plan from the entire plan"""
            
        fluents_from_executable_actions = []
        for timestep in self.data:
            timestep_fluents_from_executable_actions = []        
            for action, value in timestep.items():
                if value['part_of_plan?'] == False and value['feasible?']==True and len(value['fluents'])>0:
                    # timestep_fluents_from_executable_actions.append(value['fluents'])   
                    for fluent in value['fluents']:
                        timestep_fluents_from_executable_actions.append(fluent)
            fluents_from_executable_actions.append(timestep_fluents_from_executable_actions)
        return fluents_from_executable_actions

    def extract_fluents_from_optimal_sequence(self):
        """This function extracts the fluents from the optimal sequence"""
        
        fluents_from_optimal_sequence = []
        for timestep in self.data:
            timestep_fluents_from_optimal_sequence = []
            for action, value in timestep.items():
                if value['part_of_plan?'] == True and value['feasible?']==True and len(value['fluents'])>0:
                    # timestep_fluents_from_optimal_sequence.append(value['fluents'])   
                    for fluent in value['fluents']:
                        timestep_fluents_from_optimal_sequence.append(fluent)
            fluents_from_optimal_sequence.append(timestep_fluents_from_optimal_sequence)
        return fluents_from_optimal_sequence
    
    def print_all(self):
        print(self.optimal_sequence)
        print(self.executable_actions)
        print(self.inexecutable_actions)
        print(self.fluents_from_executable_actions)
        print(self.fluents_from_optimal_sequence)
    
# class Blocksworld(Domain_question_generation):
#     def __init__(self,data):
#         super().__init__(data)
        
#     def composite_question_1(self):
#         composite_question_answers = defaultdict(dict)
#         sequence_length = 1
        
#         for i in range(3):
#             custom_valid_action_sequences = self.optimal_sequence.copy()
#             custom_valid_action_sequences= custom_valid_action_sequences[:sequence_length]
#             select_random_inexecutable_action = random.choice(self.inexecutable_actions[i+1])
#             random_index = random.choice(range(len(custom_valid_action_sequences)+1))
#             custom_valid_action_sequences.insert(random_index,select_random_inexecutable_action)
#             if random_index == 0:
#                 sequence_before_first_infeasible_action = self.fluents_from_optimal_sequence[self.optimal_sequence.index(custom_valid_action_sequences[random_index+1])]
#             else:
#                 sequence_before_first_infeasible_action = self.fluents_from_optimal_sequence[self.optimal_sequence.index(custom_valid_action_sequences[random_index-1])]    
#             question = f"""Given the sequence of actions: {custom_valid_action_sequences},what is the state before the first infeasible action in the sequence??"""
#             composite_question_answers[question] = sequence_before_first_infeasible_action
#             sequence_length += 1
#         return composite_question_answers    
            
            
                        

In [3]:
result = Domain_question_generation('C:/Users/shri1/reasoning_about_actions/data.jsonl')
result.print_all()

['action_unstack(b2,b1)', 'action_put_down(b2)', 'action_pick_up(b1)', 'action_stack(b1,b2)']
[[], ['action_stack(b2,b1)'], [], ['action_stack(b2,b1)']]
[['action_pick_up(b2)', 'action_pick_up(b1)', 'action_stack(b1,b2)', 'action_put_down(b1)', 'action_unstack(b1,b2)', 'action_put_down(b2)', 'action_stack(b2,b1)'], ['action_pick_up(b2)', 'action_pick_up(b1)', 'action_stack(b1,b2)', 'action_put_down(b1)', 'action_unstack(b1,b2)', 'action_unstack(b2,b1)'], ['action_pick_up(b2)', 'action_stack(b1,b2)', 'action_put_down(b1)', 'action_unstack(b1,b2)', 'action_put_down(b2)', 'action_stack(b2,b1)'], ['action_pick_up(b2)', 'action_pick_up(b1)', 'action_put_down(b1)', 'action_unstack(b1,b2)', 'action_unstack(b2,b1)']]
[[], ['clear(b2)', 'ontable(b1)', 'on(b2,b1)', 'handempty'], [], ['clear(b2)', 'ontable(b1)', 'on(b2,b1)', 'handempty']]
[['ontable(b1)', 'clear(b1)', 'holding(b2)'], ['clear(b2)', 'ontable(b1)', 'ontable(b2)', 'handempty'], ['ontable(b2)', 'clear(b2)', 'holding(b1)'], ['ontable(b